In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel, mutual_info_classif
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.base import clone
from scipy.stats import entropy
from sklearn.datasets import make_classification

df = pd.read_csv('https://raw.githubusercontent.com/Pamela2710/Data-Mining/main/DataProcessing/features_completas.csv', encoding='latin-1')

# Eliminar filas con valores NaN
df = df.dropna()

df

,Media tiempo,Desviacion estandar tiempo,Varianza tiempo,Entropia tiempo,kurtosis tiempo,MSE tiempo,Tiempo en alcanzar el pico maximo en frecuencia,RMS tiempo,Peak2peak tiempo,TIEMPO peak2rms,...,solidity,std_dev,x_center_mass,x_centroid,x_centroid_cm_diff,y_center_mass,y_centroid,y_centroid_cm_diff,Etiqueta,espectrograma etiqueta
0,0.099520,0.245902,0.108890,0,0.013447,0.907095,0.115646,0.245920,0.164061,0.110557,...,1,0.410795,0.406519,0.405904,-0.000615,0.472183,0.471469,-0.000713,Long Period,espect_event_1
1,0.085819,0.876726,0.796870,0,0.050793,0.351758,0.306122,0.876697,1.000000,0.238391,...,1,12.640444,0.240467,0.239852,-0.000615,0.279601,0.278887,-0.000713,Volcano Tectonic,espect_event_2
2,0.077419,0.231336,0.099950,0,0.015484,0.882481,0.122449,0.231357,0.154656,0.130988,...,1,4.451864,0.422509,0.421894,-0.000615,0.490728,0.490014,-0.000713,Long Period,espect_event_3
3,0.102426,0.291445,0.138865,0,0.011577,0.910015,0.102041,0.291448,0.160484,0.092834,...,1,7.681243,0.418819,0.418204,-0.000615,0.486448,0.485735,-0.000713,Long Period,espect_event_4
4,0.091737,0.428982,0.247991,0,0.026587,0.723575,0.108844,0.428999,0.379571,0.141011,...,1,2.908373,0.287208,0.286593,-0.000615,0.333809,0.333096,-0.000713,Hybrid,espect_event_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,0.101867,0.214477,0.089995,0,0.018980,0.875568,0.333333,0.214480,0.163796,0.131218,...,1,2.546444,0.421894,0.421279,-0.000615,0.490014,0.489301,-0.000713,Long Period,espect_event_686
686,0.086898,0.680620,0.520007,0,0.038755,0.298080,0.455782,0.680617,0.504337,0.150002,...,1,15.365171,0.325953,0.325338,-0.000615,0.378745,0.378031,-0.000713,Volcano Tectonic,espect_event_687
687,0.077123,0.548769,0.365809,0,0.024871,0.207060,0.510204,0.548773,0.259238,0.084483,...,1,9.442264,0.306888,0.306273,-0.000615,0.356633,0.355920,-0.000713,Volcano Tectonic,espect_event_688
688,0.079244,0.720172,0.571271,0,0.031290,0.165912,0.551020,0.720172,0.407933,0.102759,...,1,14.023466,0.337638,0.337023,-0.000615,0.392297,0.391583,-0.000713,Volcano Tectonic,espect_event_689


In [ ]:
#df = df.apply(pd.to_numeric, errors='coerce')

#df_normalized = (df - df.min()) / (df.max() - df.min())

#df_normalized

# Extraer la columna "Etiqueta"
etiquetas = df['Etiqueta']

# Normalizar las columnas numéricas
cols_numericas = df.columns[:-3]  # Excluyendo las columnas "Etiqueta" y "espectrograma etiqueta"
scaler = MinMaxScaler()
df[cols_numericas] = scaler.fit_transform(df[cols_numericas])

# Unir la columna "Etiqueta" a los datos normalizados
df_normalized = pd.concat([df[cols_numericas], etiquetas], axis=1)

# Mostrar el resultado
df_normalized

,Media tiempo,Desviacion estandar tiempo,Varianza tiempo,Entropia tiempo,kurtosis tiempo,MSE tiempo,Tiempo en alcanzar el pico maximo en frecuencia,RMS tiempo,Peak2peak tiempo,TIEMPO peak2rms,...,perimeter,skewness,solidity,std_dev,x_center_mass,x_centroid,x_centroid_cm_diff,y_center_mass,y_centroid,Etiqueta
0,0.099520,0.226593,0.102626,0.0,0.013447,0.907095,0.115646,0.226602,0.163998,0.110557,...,0.750000,0.262347,0.0,0.014767,0.503906,0.503906,0.0,0.503906,0.503906,Long Period
1,0.085819,0.873569,0.795442,0.0,0.050793,0.351758,0.306122,0.873538,1.000000,0.238391,...,0.402174,0.009667,0.0,0.499546,0.152344,0.152344,0.0,0.152344,0.152344,Volcano Tectonic
2,0.077419,0.211654,0.093624,0.0,0.015484,0.882481,0.122449,0.211665,0.154592,0.130988,...,0.827640,0.046256,0.0,0.174953,0.537760,0.537760,0.0,0.537760,0.537760,Long Period
3,0.102426,0.273302,0.132812,0.0,0.011577,0.910015,0.102041,0.273296,0.160420,0.092834,...,0.874224,0.024948,0.0,0.302965,0.529948,0.529948,0.0,0.529948,0.529948,Long Period
4,0.091737,0.414361,0.242705,0.0,0.026587,0.723575,0.108844,0.414371,0.379524,0.141011,...,0.647516,0.071298,0.0,0.113770,0.251302,0.251302,0.0,0.251302,0.251302,Hybrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,0.101867,0.194363,0.083598,0.0,0.018980,0.875568,0.333333,0.194357,0.163733,0.131218,...,0.354037,0.062165,0.0,0.099423,0.536458,0.536458,0.0,0.536458,0.536458,Long Period
686,0.086898,0.672442,0.516633,0.0,0.038755,0.298080,0.455782,0.672435,0.504299,0.150002,...,0.583851,0.012588,0.0,0.607553,0.333333,0.333333,0.0,0.333333,0.333333,Volcano Tectonic
687,0.077123,0.537215,0.361351,0.0,0.024871,0.207060,0.510204,0.537213,0.259182,0.084483,...,0.591615,0.020329,0.0,0.372771,0.292969,0.292969,0.0,0.292969,0.292969,Volcano Tectonic
688,0.079244,0.713007,0.568258,0.0,0.031290,0.165912,0.551020,0.713004,0.407889,0.102759,...,0.482919,0.014551,0.0,0.554368,0.358073,0.358073,0.0,0.358073,0.358073,Volcano Tectonic


In [ ]:
df_normalized.corr()

In [ ]:
#Gain ratio - funcion objetiva
def objective_function(selected_indices):
    # Calculate Gain Ratio for the selected features
    info_gain = mutual_info_classif(X_train.iloc[:, selected_indices], Y_train, discrete_features='auto')
    intrinsic_info = [entropy(X_train.iloc[:, i]) for i in selected_indices]

    # Calculate Gain Ratio as the fitness score
    gain_ratio = [ig / ii if ii > 0 else 0 for ig, ii in zip(info_gain, intrinsic_info)]

    # Sum the Gain Ratios to get the overall fitness score (maximize Gain Ratio)
    fitness = sum(gain_ratio)

    return -fitness  # Minimize the negative fitness (maximize Gain Ratio)

#------------------------------------------------ESTA FUNCION ESTA HECHA SOLO PARA PROBAR jeje-----------------------------------------------------------
# Artificial Bee Colony algorithm
def artificial_bee_colony(objective_function, n_dim, n_iter):
    best_solution = None
    best_fitness = float('-inf')

    for _ in range(n_iter):
        # Generate random solution
        solution = np.random.randint(2, size=n_dim)

        # Evaluate solution
        fitness = objective_function(solution)

        # Update best solution
        if fitness > best_fitness:
            best_solution = solution
            best_fitness = fitness

    return best_solution

# Convertir etiquetas a números
le = LabelEncoder()
y_encoded = le.fit_transform(df_normalized['Etiqueta'])

# Dividir los datos en entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(df_normalized.drop('Etiqueta', axis=1), y_encoded, test_size=0.2, random_state=42)


# Perform feature selection using ABC
selected_features = artificial_bee_colony(objective_function, n_dim=X_train.shape[1], n_iter=100)


# Print selected features
selected_indices = np.where(selected_features)[0]
print("Selected Features:", selected_indices)


Selected Features: [  1   2   3   4   5   8   9  12  14  15  18  19  20  21  23  24  25  26
  27  29  31  32  33  35  36  37  39  40  44  49  53  55  56  57  58  60
  62  64  65  66  67  69  70  74  75  76  78  79  80  81  82  83  84  85
  86  87  88  90  91  92  94  97  98  99 103 104 106]


In [ ]:
import numpy as np

class FeatureSelectionABC:
    def __init__(self, data, labels, n_bees, max_cycles, limit):
        self.data = data
        self.labels = labels
        self.n_bees = n_bees
        self.max_cycles = max_cycles
        self.limit = limit
        self.dim = data.shape[1]  # Number of features
        self.population = self.initialize_population()
        self.fitness = np.zeros(n_bees)
        self.trial_counter = np.zeros(n_bees)
        self.best_solution = None
        self.best_fitness = -np.inf

    def initialize_population(self):
        return np.random.randint(2, size=(self.n_bees, self.dim))

    def evaluate(self, solution):
        # Placeholder for Gain Ratio evaluation
        # For simulation, returns a random fitness value for the solution
        # In practice, replace this with actual Gain Ratio computation
        solution = np.random.randint(2, size=n_dim)
        #return np.random.rand()
         # Evaluate solution
        return objective_function(solution)

    def mutate_solution(self, solution):
        idx = np.random.randint(0, self.dim)
        new_solution = solution.copy()
        new_solution[idx] = 1 - new_solution[idx]
        return new_solution

    def update_bees(self):
        for i in range(self.n_bees):
            new_solution = self.mutate_solution(self.population[i])
            new_fitness = self.evaluate(new_solution)
            if new_fitness > self.fitness[i]:
                self.population[i] = new_solution
                self.fitness[i] = new_fitness
                self.trial_counter[i] = 0
            else:
                self.trial_counter[i] += 1

    def scout_bees(self):
        for i in range(self.n_bees):
            if self.trial_counter[i] > self.limit:
                self.population[i] = np.random.randint(2, size=self.dim)
                self.fitness[i] = self.evaluate(self.population[i])
                self.trial_counter[i] = 0

    def optimize(self):
        for _ in range(self.max_cycles):
            self.update_bees()
            self.scout_bees()
            max_fitness_idx = np.argmax(self.fitness)
            if self.fitness[max_fitness_idx] > self.best_fitness:
                self.best_solution = self.population[max_fitness_idx]
                self.best_fitness = self.fitness[max_fitness_idx]


In [ ]:
#Gain ratio - funcion objetiva
def objective_function(selected_indices):
    # Calculate Gain Ratio for the selected features
    info_gain = mutual_info_classif(X_train.iloc[:, selected_indices], Y_train, discrete_features='auto')
    intrinsic_info = [entropy(X_train.iloc[:, i]) for i in selected_indices]

    # Calculate Gain Ratio as the fitness score
    gain_ratio = [ig / ii if ii > 0 else 0 for ig, ii in zip(info_gain, intrinsic_info)]

    # Sum the Gain Ratios to get the overall fitness score (maximize Gain Ratio)
    fitness = sum(gain_ratio)

    return -fitness  # Minimize the negative fitness (maximize Gain Ratio)

In [ ]:
import numpy as np
from sklearn.feature_selection import mutual_info_classif
from scipy.stats import entropy

class FeatureSelectionABC:
    def __init__(self, data, labels, n_bees, max_cycles, limit, X_train, Y_train):
        self.data = data
        self.labels = labels
        self.n_bees = n_bees
        self.max_cycles = max_cycles
        self.limit = limit
        self.dim = data.shape[1]  # Number of features
        self.population = self.initialize_population()
        self.fitness = np.zeros(n_bees)
        self.trial_counter = np.zeros(n_bees)
        self.best_solution = None
        self.best_fitness = -np.inf
        self.X_train = X_train
        self.Y_train = Y_train

    def initialize_population(self):
        return np.random.randint(2, size=(self.n_bees, self.dim))

    def evaluate(self, solution):
        return self.objective_function(solution)

    def objective_function(self, selected_indices):
        # Calculate Gain Ratio for the selected features
        info_gain = mutual_info_classif(self.X_train.iloc[:, selected_indices], self.Y_train, discrete_features='auto')
        intrinsic_info = [entropy(self.X_train.iloc[:, i]) for i in selected_indices]

        # Calculate Gain Ratio as the fitness score
        gain_ratio = [ig / ii if ii > 0 else 0 for ig, ii in zip(info_gain, intrinsic_info)]

        # Sum the Gain Ratios to get the overall fitness score (maximize Gain Ratio)
        fitness = sum(gain_ratio)

        return -fitness  # Minimize the negative fitness (maximize Gain Ratio)

    def mutate_solution(self, solution):
        idx = np.random.randint(0, self.dim)
        new_solution = solution.copy()
        new_solution[idx] = 1 - new_solution[idx]
        return new_solution

    def update_bees(self):
        for i in range(self.n_bees):
            new_solution = self.mutate_solution(self.population[i])
            new_fitness = self.evaluate(new_solution)
            if new_fitness > self.fitness[i]:
                self.population[i] = new_solution
                self.fitness[i] = new_fitness
                self.trial_counter[i] = 0
            else:
                self.trial_counter[i] += 1

    def scout_bees(self):
        for i in range(self.n_bees):
            if self.trial_counter[i] > self.limit:
                self.population[i] = np.random.randint(2, size=self.dim)
                self.fitness[i] = self.evaluate(self.population[i])
                self.trial_counter[i] = 0

    def optimize(self):
        for _ in range(self.max_cycles):
            self.update_bees()
            self.scout_bees()
            max_fitness_idx = np.argmax(self.fitness)
            if self.fitness[max_fitness_idx] > self.best_fitness:
                self.best_solution = self.population[max_fitness_idx]
                self.best_fitness = self.fitness[max_fitness_idx]

# Ejemplo de cómo utilizar la clase FeatureSelectionABC

# Supongamos que tienes los datos de entrenamiento X_train e Y_train
# Aquí se inicializa y se ejecuta el algoritmo ABC para selección de características
# Asegúrate de proporcionar X_train e Y_train al inicializar la clase FeatureSelectionABC

# feature_selection = FeatureSelectionABC(data, labels, n_bees, max_cycles, limit, X_train, Y_train)
# feature_selection.optimize()


# Convertir etiquetas a números
le = LabelEncoder()
y_encoded = le.fit_transform(df_normalized['Etiqueta'])

# Dividir los datos en entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(df_normalized.drop('Etiqueta', axis=1), y_encoded, test_size=0.2, random_state=42)

# Example usage:
# For the purpose of this example, we'll simulate the data and labels
np.random.seed(42) # For reproducibility
data = np.random.rand(100, 10) # 100 samples, 10 features
labels = np.random.randint(0, 2, 100) # Binary labels

abc = FeatureSelectionABC(data, labels, n_bees, max_cycles, limit, X_train, Y_train)
abc.optimize()

print("Best solution:", abc.best_solution)
print("Best fitness:", abc.best_fitness)


AttributeError: 'FeatureSelectionABC' object has no attribute 'X_train'

# Literatura

## Algoritmo ABC

- Karaboga, Dervis, and Bahriye Basturk. "Artificial Bee Colony (ABC) Optimization Algorithm for Solving Constrained Optimization Problems." Erciyes University Engineering Faculty, Department of Computer Engineering, 2007. [Paper](https://sci-hub.se/https://doi.org/10.1007/978-3-540-72950-1_77)
- [Visualizacion en YT](https://www.youtube.com/watch?v=3qQr1eZwz5E)
- [Popular packet implementation de ABC](https://pypi.org/project/beecolpy/)

